# A brief introduction to data issues one might encounter

The Adult dataset:  https://archive.ics.uci.edu/dataset/2/adult

This was part of our first assignment.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import ipywidgets

from ucimlrepo import fetch_ucirepo

## Importing the data

In [ ]:
# fetch dataset 
adult = fetch_ucirepo(id=2) 

# let's start with the original data
data = adult.data.original

# data (as pandas dataframes) 
# x = adult.data.features 
# y = adult.data.targets 
  
# # metadata 
# print(adult.metadata) 
  
# # variable information 
# print(adult.variables) 


## Figuring out what we're working with

In [ ]:
data.info()

In [ ]:
data.head()

## What are our features? and what is our target?

In [ ]:
data['income'].value_counts()

Wait!
  * redundant target values?
  * imbalanced data?

In [ ]:
data['income_binary'] = data['income'].map({'<=50K':0,
                                            '<=50K.':0,
                                            '>50K':1,
                                            '>50K.':1})

In [ ]:
data['income_binary'].value_counts()

In [ ]:
data['income_binary'].value_counts(normalize=True)

In [ ]:
def printuniq(c = 'age'):
    print(data[c].sort_values().unique())

ipywidgets.interact(printuniq, c=ipywidgets.Dropdown(options=data.columns));

## How do the features play out against the target?

In [ ]:
sns.histplot(data=data, 
             y='workclass', 
             hue='income')

In [ ]:
sns.histplot(data=data, 
             y='workclass', 
             hue='income_binary')

In [ ]:
sns.histplot(data=data, 
             y='workclass', 
             hue='income_binary',
             multiple='stack')

In [ ]:
sns.histplot(data=data, 
             y='workclass', 
             hue='income_binary',
             multiple='fill')

In [ ]:
sns.histplot(data=data, 
             y='sex', 
             hue='income_binary',
             multiple='dodge',shrink=0.8)

In [ ]:
sns.countplot(data=data, 
             y='sex', 
             hue='income_binary')

In [ ]:
def varcountplot(c='age'):
    sns.histplot(data=data, 
                 y=c, 
                 hue='income_binary',
                multiple='fill')

ipywidgets.interactive(varcountplot,c=data.columns)

In [ ]:
def varcountplot(c='age'):
    sns.histplot(data=data, 
                 y=c, 
                 hue='income_binary',
                multiple='stack')

ipywidgets.interactive(varcountplot,c=data.columns)

In [ ]:
data['capital-loss'].value_counts()

In [ ]:
data['capital-gain'].value_counts()

In [ ]:
data['native-country'].value_counts()

# Binning

In [ ]:
data['age'].hist()

In [ ]:
# These are the quartiles
data['age'].describe()

### Version 1: using pd.cut



In [ ]:
bins = [17.0, 28.0, 37.0, 48.0, 90.0]
labels = ['17-28', '28-37', '37-48', '48-90']

data['ageCat1'] = pd.cut(data['age'],
                         bins=bins,
                         labels=labels)

In [ ]:
sns.countplot(data=data, 
              x='ageCat1', 
              hue='income_binary')

### Version 2: using pd.qcut with quartiles



In [ ]:
labels = ['17-28', '28-37', '37-48', '48-90']

data['ageCat2'] = pd.qcut(data['age'], q=4, labels=labels)

In [ ]:
sns.countplot(data=data, 
              x='ageCat2',
              hue='income_binary')

Note that the describe() method gives accessible values too:

### Version 3: using pd.cut and "describe" output

In [ ]:
bins = data['age'].describe()[['min','25%','50%','75%','max']].values
labels = ['17-28', '28-37', '37-48', '48-90']

data['ageCat3'] = pd.cut(data['age'], bins=bins, labels=labels)

In [ ]:
data[['age','ageCat1','ageCat2','ageCat3']]

In [ ]:
sns.countplot(data=data, x='ageCat3',hue='income_binary')

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
data[['age']]

In [ ]:
binning_model = KBinsDiscretizer(n_bins=4, encode='ordinal')
# binning_model = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')

data['age_binned'] = binning_model.fit_transform(data[['age']])
binning_model.bin_edges_

In [ ]:
data[['age', 'age_binned']].sample(10)

In [ ]:
data['age_binned'].hist()

## Looking for errors, junk, or simply discardees

In [ ]:
data['capital-gain'].value_counts()

In [ ]:
data['capital-loss'].value_counts()

Lots of zeros -> may decide to drop.

In [ ]:
data.drop(['capital-gain','capital-loss'], axis=1, inplace=True)

In [ ]:
data.info()

Most records are for the United States, so maybe you decide to only keep that value for native country
* drop records that don't satisfy a condition

In [ ]:
data = data.loc[data['native-country']=='United-States']

In [ ]:
data = data.drop(['native-country'], axis=1)

In [ ]:
data.info()

## Clean up a bit to collect your features and targets

In [ ]:
x = data[['age','workclass','education',
             'occupation','race','sex','hours-per-week']]
y = data['income_binary']

In [ ]:
x.tail()

### be careful about index continuity

In [ ]:
x.reset_index(inplace=True, drop=True)

In [ ]:
x.tail()

In [ ]:
y.tail()

In [ ]:
y.reset_index(inplace=True, drop=True)

In [ ]:
y.tail()

## Checking out illogical entries, duplicates, outliers

In [ ]:
sns.countplot(data=data, x='relationship', hue='sex')

In [ ]:
data.loc[((data['relationship'] == 'Husband') & (data['sex'] == 'Female')) | 
           ((data['relationship'] == 'Wife') & (data['sex'] == 'Male'))]

In [ ]:
data.shape

In [ ]:
data = data.loc[~(((data['relationship'] == 'Husband') & (data['sex'] == 'Female')) |
                  ((data['relationship'] == 'Wife') & (data['sex'] == 'Male')))]

In [ ]:
data.shape

# Outliers
Any values need to be clipped?
* this version is pretty sanitary, but we can impose limits for example's sake

In [ ]:
data.describe(include='all')

In [ ]:
data['hours-per-week'].hist(bins=50)

In [ ]:
data[['hours-per-week']].boxplot()

In [ ]:
# total number of hours worked for 7 days a week at 12 hrs/day
12*7

In [ ]:
data['hours-per-week'] = data['hours-per-week'].clip(lower=20, 
                                                     upper=84)

In [ ]:
data[['hours-per-week']].boxplot()

In [ ]:
data['hours-per-week'].hist(bins=50)

### Duplicate checking!

In [ ]:
data[data.duplicated(keep=False)].head()

In [ ]:
data[data.duplicated(keep=False)].sort_values(by='fnlwgt').head(6)

In [ ]:
data.shape

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

# Normalization

Scaling can be very important for algorithms that are sensitive to distance-based metrics
* MinMax
  * new scale: [0,1]
* Standard Scaling
  * new scale: mean 0, standard deviation 1
  * $z = (x - \mu)/\sigma$

In [ ]:
data[['hours-per-week']].boxplot()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
# scaler = MinMaxScaler()
scaler = StandardScaler()

In [ ]:
scaler.fit(data[['hours-per-week']])
tmphrs = scaler.transform(data[['hours-per-week']])
plt.boxplot(tmphrs)

In [ ]:
data[['hours-per-week']].boxplot()

## Missing data

In [ ]:
data.columns

In [ ]:
data['workclass'].unique()

In [ ]:
data.loc[data['workclass']=='?', ['occupation']].value_counts()

In [ ]:
data.loc[data['occupation']=='?', ['workclass']].value_counts()

In [ ]:
data.loc[(data['occupation']=='?') |
    (data['workclass']=='?'), 'workclass'].value_counts()

In [ ]:
data.groupby(['workclass'])['income_binary'].value_counts()

In [ ]:
data['workclass'].value_counts()

In [ ]:
data['workclass'].isna().sum()

In [ ]:
data['occupation'].isna().sum()

In [ ]:
data['occupation'].unique()

In [ ]:
data['sex'].isna().sum()

# ... and so on.  One must be thorough before the fun ML comes.